In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [19]:
!pip install nltk scikit-learn pandas matplotlib


In [25]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development.

In [9]:

import pandas as pd

url_test = "https://raw.githubusercontent.com/KJanzon/lab-natural-language-processing/refs/heads/main/data/kg_test.csv"
url_train = "https://raw.githubusercontent.com/KJanzon/lab-natural-language-processing/refs/heads/main/data/kg_train.csv"


In [13]:
## Read Data for the Fraudulent Email Kaggle Challenge
train_data = pd.read_csv(url_train)
test_data = pd.read_csv(url_test)

# Reduce the training set to speed up development.
# Modify for final system
train_data = train_data.head(1000)
print(train_data.shape)
train_data.fillna("",inplace=True)
print(train_data.head)

(1000, 2)
<bound method NDFrame.head of                                                   text  label
0    DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                             Will do.      0
2    Nora--Cheryl has emailed dozens of memos about...      0
3    Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                  fyi      0
..                                                 ...    ...
995  So what's the latest? It sounds contradictory ...      0
996  TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...      1
997  Barb I will call to explain. Are you back in t...      0
998    Yang on travelNot free tonite.May work tomorrow      0
999  sbwhoeopSunday February 21 2010 7:42 PMHShaunH...      0

[1000 rows x 2 columns]>


<ipython-input-13-15ba5bf19852>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.fillna("",inplace=True)


### Let's divide the training and test set into two partitions

In [14]:
# Ensuring the dataset has the expected structure
if "text" in train_data.columns and "label" in train_data.columns:
    X = train_data["text"]  # Features (SMS messages)
    y = train_data["label"]  # Labels (Spam or Ham)
else:
    X = train_data.iloc[:, 0]  # Assuming first column is text
    y = train_data.iloc[:, 1]  # Assuming second column is label


## Data Preprocessing

In [26]:
# Download NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [27]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [28]:
import re

def clean_html(text):
    # Remove inline JavaScript & CSS
    text = re.sub(r'<script.*?</script>', '', text, flags=re.DOTALL)  # Remove JavaScript
    text = re.sub(r'<style.*?</style>', '', text, flags=re.DOTALL)  # Remove CSS

    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)

    # Remove remaining HTML tags
    text = re.sub(r'<.*?>', '', text)

    return text

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters

- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [ ]:
# Your code

## Now let's work on removing stopwords
Remove the stopwords.

In [ ]:
# Your code

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [ ]:
# Your code

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [ ]:
# Your code

## Extra features

In [ ]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x))

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x))

data_train.head()

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code

## And the Train a Classifier?

In [ ]:
# Your code

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code